In [1]:
# Necessary Imports
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Enable Eager Execution
tf.enable_eager_execution()

print("Tensorflow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

Tensorflow version: 1.8.0
Eager execution: True


In [14]:
# Load MNIST data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
X=mnist.train.images
Y=mnist.train.labels[:,0].reshape(55000, 1)
print(X.shape)
print(Y.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 1)


In [31]:
# Convert to tensorflow tensors
X = tf.convert_to_tensor(X)
Y = tf.convert_to_tensor(Y)
# Contruct tensorflow dataset object from tensors
tfds = tf.data.Dataset.from_tensors((X,Y))
# View a single example entry from a batch
features, label = iter(tfds).next()
print("example features: ", features[0])
print("example label: ", label[0])

example features:  tf.Tensor(
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0. 

In [32]:
# Simple Logistic Regression Model (Sigmoid)
sig_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(784,))
])

In [33]:
# Define forward propogation
# loss function to be binary crossentropy
def forward_prop(model, x, y):
    a = model(x)
    loss = tf.losses.sigmoid_cross_entropy(labels=y,logits=a)
    return loss

In [38]:
# Define backward propogation
# Object Gradient Tape does the magic of calculating gradients
#tape = tf.GradientTape()
def backward_prop(model, tape, loss):
    grad = tape.gradient(loss, model.variables)
    return grad

In [54]:
# Define optimizer
lr = 1
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [55]:
# The Big For-Loop Training
m = 55000
dc = tf.constant(1/m)
num_epochs = 100

for epoch in range(num_epochs):
    loss_avg = tfe.metrics.Mean()
    batch_grad = None
    
    for x,y in tfds:
        with tf.GradientTape(persistent=True) as tape:
            loss = forward_prop(sig_model, x, y)
        grad = backward_prop(sig_model, tape, loss)
        loss_avg(loss)
        optimizer.apply_gradients(zip(grad, sig_model.variables))
    
    print("Loss: {}".format(loss_avg.result()))

Loss: 0.6979889273643494
Loss: 0.6975814700126648
Loss: 0.6972430348396301
Loss: 0.6969552636146545
Loss: 0.6967069506645203
Loss: 0.6964920163154602
Loss: 0.6963039636611938
Loss: 0.6961367726325989
Loss: 0.6959872245788574
Loss: 0.6958532929420471
Loss: 0.6957332491874695
Loss: 0.695622444152832
Loss: 0.6955223083496094
Loss: 0.6954305171966553
Loss: 0.6953459978103638
Loss: 0.6952683329582214
Loss: 0.6951958537101746
Loss: 0.6951286792755127
Loss: 0.6950662136077881
Loss: 0.6950088143348694
Loss: 0.6949539184570312
Loss: 0.6949029564857483
Loss: 0.6948545575141907
Loss: 0.6948087215423584
Loss: 0.6947664618492126
Loss: 0.6947264671325684
Loss: 0.6946877837181091
Loss: 0.6946513056755066
Loss: 0.6946169137954712
Loss: 0.6945834755897522
Loss: 0.6945523619651794
Loss: 0.69452303647995
Loss: 0.6944941878318787
Loss: 0.6944671273231506
Loss: 0.6944410800933838
Loss: 0.6944153904914856
Loss: 0.6943914890289307
Loss: 0.6943686604499817
Loss: 0.6943466067314148
Loss: 0.6943265795707703
Los